In [1]:
import json
import os
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alpha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import re

def clean_str(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '[^\w\s.?!]'         # 특수기호제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '\s+'         # tab to whitespace
    text = re.sub(pattern=pattern, repl=' ', string=text)
    return text  

# 1. Language model data 생성

In [3]:
file_names = []
for root, dirs, files in os.walk('data/petitions/2018'):
    for fname in files:
        full_fname = os.path.join(root, fname)

        file_names.append(full_fname)

In [4]:
out_file_path = "data/petitions/petition.lm.train"
with open(out_file_path, 'a', encoding='UTF8') as out_file:
    for file_name in file_names:
        with open(file_name, "r", encoding='utf-8', errors='ignore') as in_file:
            datas = in_file.readlines()
            
            for i in range(len(datas)):
                data = json.loads(datas[i])

                content = clean_str(data['content'])

                sentences = sent_tokenize(content)

                # BERT 모델은 문단 내 문장의 개수가 2개 이상이여야 함
                if len(sentences) < 2:
                    continue

                for sentence in sentences:
                    sentence = sentence.strip()

                    out_file.write(sentence + "\n")

                if i != 0 and i % 1000 == 0:
                    print(i, "items has been converted")

                out_file.write("\n")

                #if i == 5000:
                #    break

            print("Total {0} items has been converted".format(len(datas)))

2000 items has been converted
3000 items has been converted
4000 items has been converted
6000 items has been converted
8000 items has been converted
9000 items has been converted
10000 items has been converted
11000 items has been converted
12000 items has been converted
13000 items has been converted
14000 items has been converted
15000 items has been converted
17000 items has been converted
18000 items has been converted
19000 items has been converted
20000 items has been converted
21000 items has been converted
22000 items has been converted
24000 items has been converted
29000 items has been converted
Total 31247 items has been converted
2000 items has been converted
3000 items has been converted
6000 items has been converted
7000 items has been converted


KeyboardInterrupt: 

# 2. Wordpiece model 생성

In [19]:
import sentencepiece as spm

In [20]:
input_file = 'data/petitions/petition.lm.train'
templates = '--input={} --model_prefix={} --vocab_size={}'
vocab_size = 30000
prefix = 'data/petitions/sp-30000'
cmd = templates.format(input_file, prefix, vocab_size)

In [21]:
# train
spm.SentencePieceTrainer.Train(cmd)

True

# 3. 사전 생성

In [22]:
in_file_path = "data/petitions/sp-30000.vocab"
out_file_path = "data/petitions/vocab.txt"

with open(in_file_path, encoding='utf-8') as f_in:
    with open(out_file_path, 'a', encoding='UTF8') as f_out:
        f_out.write("[PAD]\n")
        f_out.write("[CLS]\n")
        f_out.write("[SEP]\n")
        f_out.write("[MASK]\n")
        for word in f_in:
            if word.split('\t')[0].strip() == '<unk>':
                f_out.write("[UNK]\n")
            else:
                f_out.write(word.split('\t')[0].strip() + "\n")

# 4 Multi-classification model data 생성

In [20]:
file_names = []
for root, dirs, files in os.walk('data/petitions/all'):
    for fname in files:
        full_fname = os.path.join(root, fname)

        file_names.append(full_fname)

In [21]:
c_list = []
for i, file_name in enumerate(file_names):
    with open(file_name, "r", encoding='utf-8', errors='ignore') as in_file:
        datas = in_file.readlines()

        for j in range(len(datas)):
            data = json.loads(datas[j])
            content = clean_str(data['content'])
            category = data['category']
            begin = data['begin']
            c_list.append((content, category, begin))
            
    print("{0} file has been converted".format(i+1))
print("Total {0} datas has been converted".format(len(c_list)))

1 file has been converted
2 file has been converted
3 file has been converted
4 file has been converted
5 file has been converted
6 file has been converted
7 file has been converted
8 file has been converted
9 file has been converted
10 file has been converted
11 file has been converted
12 file has been converted
13 file has been converted
14 file has been converted
15 file has been converted
16 file has been converted
17 file has been converted
18 file has been converted
19 file has been converted
20 file has been converted
21 file has been converted
22 file has been converted
23 file has been converted
24 file has been converted
25 file has been converted
Total 436660 datas has been converted


In [22]:
import pandas as pd
from sklearn.utils import shuffle

In [23]:
df = pd.DataFrame(c_list, columns=['contents','category','begin']) 

In [24]:
df.head(5)

,contents,category,begin
0,안녕하세요. 현재 사대 교대 등 교원양성학교들의 예비교사들이 임용절벽에 매우 힘들어...,육아/교육,2017-08-25
1,서울시에서는 10년부터 정수지와 배수지 청소용 화학제품인 세정제에 대하여 첨부 와 ...,안전/환경,2017-08-25
2,얼마 전 살충제 계란 파동 등으로 씨끄러웠던 중 친환경업체한살림 등의 DDT파동으로...,기타,2017-08-25
3,길고양이들 밥주고있는 사람입니다. 최근에 동네주민과 트러블이 생겨 싸움이 일어났습니...,반려동물,2017-08-25
4,안녕하세요 지인분이 성관계 몰래카메라가 유출되어 방송통신 심의 위원회에 접수해서 처...,인권/성평등,2017-08-25


In [25]:
len(df)

436660

In [26]:
df.groupby('category')['category'].count()

category
경제민주화          17653
교통/건축/국토       30301
기타             54465
농산어촌            2121
문화/예술/체육/언론    19777
미래             20265
반려동물            4507
보건복지           27139
성장동력            7798
안전/환경          35468
외교/통일/국방       29002
육아/교육          27426
인권/성평등         39034
일자리            25016
저출산/고령화대책       4001
정치개혁           70536
행정             22151
Name: category, dtype: int64

In [27]:
# 기타 항목 제거
df = df.loc[df['category'] != '기타']

In [28]:
len(df)

382195

In [29]:
'''category_dictionary = {
    '경제민주화' : '0', 
    '교통/건축/국토' : '1', 
    '농산어촌' : '2',
    '문화/예술/체육/언론' : '3',
    '미래' : '4',
    '반려동물' : '5',
    '보건복지' : '6',
    '성장동력' : '7',
    '안전/환경' : '8',
    '외교/통일/국방' : '9',
    '육아/교육' : '10',
    '인권/성평등' : '11',
    '일자리' : '12',
    '저출산/고령화대책' : '13',
    '정치개혁' : '14',
    '행정' : '15',
    '기타' : '16',
} '''

category_dictionary = {
    '경제민주화' : '0', 
    '교통/건축/국토' : '1', 
    '농산어촌' : '2',
    '문화/예술/체육/언론' : '3',
    '미래' : '4',
    '반려동물' : '5',
    '보건복지' : '6',
    '성장동력' : '7',
    '안전/환경' : '8',
    '외교/통일/국방' : '9',
    '육아/교육' : '10',
    '인권/성평등' : '11',
    '일자리' : '12',
    '저출산/고령화대책' : '13',
    '정치개혁' : '14',
    '행정' : '15',
} 

In [30]:
df['category_label'] = df['category'].map(category_dictionary)

In [31]:
df.head(5)

,contents,category,begin,category_label
0,안녕하세요. 현재 사대 교대 등 교원양성학교들의 예비교사들이 임용절벽에 매우 힘들어...,육아/교육,2017-08-25,10
1,서울시에서는 10년부터 정수지와 배수지 청소용 화학제품인 세정제에 대하여 첨부 와 ...,안전/환경,2017-08-25,8
3,길고양이들 밥주고있는 사람입니다. 최근에 동네주민과 트러블이 생겨 싸움이 일어났습니...,반려동물,2017-08-25,5
4,안녕하세요 지인분이 성관계 몰래카메라가 유출되어 방송통신 심의 위원회에 접수해서 처...,인권/성평등,2017-08-25,11
5,대통령님 민주주의 사회에서는 국공립과 사립은 공존하여 서로 경쟁 발전해야하며 시급한...,육아/교육,2017-08-25,10


In [32]:
df["contents_length"]= df["contents"].str.len() 

In [33]:
# contens 갯수 확인 후 제거
df = df.loc[(df['contents_length'] > 20) & (df['contents_length'] < 30000)] 

In [34]:
df.groupby('category')['category'].count()

category
경제민주화          17144
교통/건축/국토       29739
농산어촌            2061
문화/예술/체육/언론    18984
미래             18995
반려동물            4326
보건복지           26364
성장동력            7423
안전/환경          33964
외교/통일/국방       28065
육아/교육          26363
인권/성평등         36811
일자리            24459
저출산/고령화대책       3930
정치개혁           61882
행정             21564
Name: category, dtype: int64

In [35]:
df.sort_values(by=['contents_length'])

,contents,category,begin,category_label,contents_length
29210,인면 수심의 조두순 출소를 반대합니다.,인권/성평등,2017-11-10,11,21
28063,인간도 아닌 놈 출소 절대 반대합니다.,인권/성평등,2017-11-09,11,21
122234,북 김영철 방남 허락은 제발 아주세요.,외교/통일/국방,2018-02-22,9,21
227068,외국인불법고용주 신고포상금제 청원합니다,일자리,2018-07-05,12,21
162877,현재 국회의원도 전수조사 해야 합니다.,정치개혁,2018-04-17,14,21
...,...,...,...,...,...
307863,저희 아버지의 억울함을 풀어주세요문화재로 인한 국가의 세금갑질 분야 경제 민주화대...,경제민주화,2018-10-10,0,28295
235017,대한민국 문화계 체육계 언론계 관련 모든 관계자 여러분 제 작성한 이 긴 청원글을 ...,문화/예술/체육/언론,2018-07-14,3,28851
355845,2017 우리나라 합계 출산율 1.05명 강신욱 통계청장은 최근 2018년 합계출산...,정치개혁,2018-12-10,14,28871
150077,우리가 잊어버리고 있는 질문 한 개 지 올리겠습니다. 왜 투표을 하시나요? 곰곰히 ...,미래,2018-04-01,4,29223


In [36]:
df = shuffle(df)

In [37]:
df.head(5)

,contents,category,begin,category_label,contents_length
289026,입법도 없고 . 사법도 없고. 행정도 없고. 오로지 서민 갉아먹는 금수들의 세상. ...,미래,2018-09-12,4,78
227963,대우조선노조현대자동차 노조.도대체 얼마를 받고일하길래.맨날 임금올려달라고 투쟁이나하...,일자리,2018-07-05,12,273
171122,지상파 방송과 종편채널 모두가 하루종이 문대통령과 김정은의 대한 이얘기도 채널들이 ...,인권/성평등,2018-04-27,11,180
427736,IMF 권고를 뛰어넘는 담대한 BOLD 추경을 하루 속히 편성 집행하라! 2019년...,성장동력,2019-03-21,7,1015
132067,아시다시피 현 수능국어에는 파트가 5개입니다. 화법작문문법문학독서 이렇게요. 그런데...,육아/교육,2018-03-06,10,1128


In [68]:
df_fair = df.loc[(df['category_label'] == '2') | (df['category_label'] == '5') | (df['category_label'] == '7') | (df['category_label'] == '13')] 

In [69]:
len(df_fair)

17740

In [70]:
df_fair = df_fair.append(df.loc[(df['category_label'] == '0')].iloc[:7688])
df_fair = df_fair.append(df.loc[(df['category_label'] == '1')].iloc[:7688])
df_fair = df_fair.append(df.loc[(df['category_label'] == '3')].iloc[:7688])
df_fair = df_fair.append(df.loc[(df['category_label'] == '4')].iloc[:7688])
df_fair = df_fair.append(df.loc[(df['category_label'] == '6')].iloc[:7688])
df_fair = df_fair.append(df.loc[(df['category_label'] == '8')].iloc[:7688])
df_fair = df_fair.append(df.loc[(df['category_label'] == '9')].iloc[:7688])
df_fair = df_fair.append(df.loc[(df['category_label'] == '10')].iloc[:7688])
df_fair = df_fair.append(df.loc[(df['category_label'] == '11')].iloc[:7688])
df_fair = df_fair.append(df.loc[(df['category_label'] == '12')].iloc[:7688])
df_fair = df_fair.append(df.loc[(df['category_label'] == '14')].iloc[:7688])
df_fair = df_fair.append(df.loc[(df['category_label'] == '15')].iloc[:7688])

In [71]:
len(df_fair)

109996

In [72]:
df_fair.groupby('category')['category'].count()

category
경제민주화          7688
교통/건축/국토       7688
농산어촌           2061
문화/예술/체육/언론    7688
미래             7688
반려동물           4326
보건복지           7688
성장동력           7423
안전/환경          7688
외교/통일/국방       7688
육아/교육          7688
인권/성평등         7688
일자리            7688
저출산/고령화대책      3930
정치개혁           7688
행정             7688
Name: category, dtype: int64

In [77]:
df_fair = shuffle(df_fair)

In [78]:
df_dev = df_fair.iloc[:10000]
df_train = df_fair.iloc[10000:]

In [79]:
print(len(df_train))
print(len(df_dev))
print(len(df_train)+len(df_dev))

99996
10000
109996


In [80]:
df_train.groupby('category')['category'].count()

category
경제민주화          6985
교통/건축/국토       6952
농산어촌           1863
문화/예술/체육/언론    6976
미래             6990
반려동물           3955
보건복지           6952
성장동력           6788
안전/환경          6977
외교/통일/국방       7033
육아/교육          6965
인권/성평등         7013
일자리            7014
저출산/고령화대책      3562
정치개혁           6993
행정             6978
Name: category, dtype: int64

In [81]:
df_dev.groupby('category')['category'].count()

category
경제민주화          703
교통/건축/국토       736
농산어촌           198
문화/예술/체육/언론    712
미래             698
반려동물           371
보건복지           736
성장동력           635
안전/환경          711
외교/통일/국방       655
육아/교육          723
인권/성평등         675
일자리            674
저출산/고령화대책      368
정치개혁           695
행정             710
Name: category, dtype: int64

In [82]:
train_file_path = "data/petitions/petition.mcc.fair.train"
dev_file_path = "data/petitions/petition.mcc.fair.dev"
df_train.to_csv(train_file_path, columns=['contents','category_label'],sep='\t')
df_dev.to_csv(dev_file_path, columns=['contents','category_label'],sep='\t')